In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import re 
from numpy import NaN

In [3]:
df = pd.read_csv('welcome.csv', encoding="latin1")

In [4]:
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [5]:
print(df)

                            PMID/PMCID  \
0                                  NaN   
1                           PMC3679557   
2                23043264  PMC3506128    
3                  23438330 PMC3646402   
4                 23438216 PMC3601604    
5                           PMC3579457   
6                           PMC3709265   
7                 23057412 PMC3495574    
8                    PMCID: PMC3780468   
9                    PMCID: PMC3621575   
10                   PMCID: PMC3739413   
11                   PMCID: PMC3530961   
12                   PMCID: PMC3624797   
13                          PMC3413243   
14                          PMC3694353   
15                          PMC3572711   
16                            22610094   
17                   PMCID: PMC3586974   
18        23455506  PMCID: PMC3607399    
19          PMID: 24015914 PMC3833349    
20                       : PMC3805332    
21                                 NaN   
22            PMCID:\n    PMC36567

#1. What are the five most common journals and the total articles for each?
#2. Count the number of journals 
#3. Organize the journal's names by common name

In [6]:
df = df.rename(columns={'Journal title': 'Journal_Title', 
                        'Article title': 'Article_Title',
                       'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost',
                       'Unnamed: 5': 'Unnamed'})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID       1928 non-null object
Publisher        2127 non-null object
Journal_Title    2126 non-null object
Article_Title    2127 non-null object
Cost             2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [8]:
# Data cleaning for Publishers column

publisher_changes = {
'ACS': ['ACS (Amercian Chemical Society) Publications',
           'ACS Publications',
           'American Chemical Society',
           'AMERICAN CHEMICAL SOCIETY',
           'American Chemical Society Publications'],
    'American Society for Biochemistry and Molecular Biology':
        ['American Society for Biochemistry and Molecular Biolgy',
        'American Soc for Biochemistry and Molecular Biology',
        'The American Society for Biochemistry and Molecular Biology',
        'The American Society for Biochemistry and Molecular Biology, Inc'],
    'American Society of Hematology': ['American Society of Haematology',
                                       'American Society of Hamatology'],
    'ASBMB': ['ASBMB Cadmus',
             'ASBMB/Cadmus',
             'ASBMB/Cenveo Publisher Services',
             'ASBMC /CENVEO'],
    'ASM': ['ASM (American Society for Microbiology)'],
    'Bentham Science Publishers': ['Benthan Science Publishers'],
    'BioMed Central': ['BioMed Central Limited',
                      'BioMed Central Ltd'],
    'BioScientific': ['Bioscientifica'],
    'BMJ Group': ['BMJ',
                 'BMJ Publishing Group',
                 'BMJ PUBLISHING GROUP',
                 'BMJ Publishing Group Ltd',
                 'BMJ Publishing Group Ltd & British Thoracic Society'],
    'Cadmus Journal Services': ['Cadmus',
                               'CADMUS JOURNAL SERVICE',
                               'CADMUS JOURNAL SERVICES'],
    'Cambridge University Press': ['Cambridge Univ Press'],
    'Cenveo Publisher services': ['Cenveo Publisher Services/ASM JV1'],
    'Cold Spring Harbor Press': ['Cold Spring Habour Press',
                                'Cold Spring Harbor',
                                'Cold Spring Harbor Laboratory Press',
                                'Cold Spring Harbor Publications'],
    'Company of Biologists': ['Company of Biologist',
                             'Company of Biologists Ltd',
                             'THE COMPANY OF BIOLOGISTS',
                             'The company of Biolgists',
                             'The Company of Biologists',
                             'The Company of Biologists Ltd'],
    'Copyright Clearace Center': ['Copyright Clearance Center'],
    'Dartmouth Journal Services': ['Darmouth Journal Services',
                                  'Dartmouth Journals'],
    'Elsevier': ['Elseveier Science', 
                 'ELSEVIER', 
                 'Elsevier (Cell Press)',
                 'Elsevier / Cell Science', 
                 'Elsevier/Cell Press', 
                 'Elsevier Ltd',
                 'Elsevier B.V.'],
    'European Society of Endocrinolog': ['European Society of Endocrinology'],
    'Federation of American Societies for Experimental Biology': 
            ['Federation of American Societies for Experimental Biology (FASEB)'],
    'Frontiers': ['Frontiers Media', 'Frontiers Media SA'],
    'Future Medicine': ['Future Medicine Ltd'],
    'Hindawi Publishing Corporation': ['Hindawi'],
    'Impact Journals': ['Impact Journals LLC'],
    'Informa Healthcare': ['Informa Healthcare communications'],
    'International Union Against Tuberculosis and Lung Disease':
            ['International Union Against tuberculosis and Lung Disease'],
    'International Union of Crystallography': 
        ['International Union of Crystallography (iucr)'],
    'John Wiley & Sons': ['John Wiley', 
                          'JOHN WILEY & SONS',
                          'John Wiley & Sons Inc', 
                          'John Wiley & Sons Ltd',
                          'John Wiley & Sons, Inc. '],
    'Karger': ['KARGER'],
    'Landes Bioscience': ['Landes Biosciences'],
    'Mary Ann Liebert': ['MARY ANN LIEBERT INC', 
                         'Mary Ann Liebert, Inc. Publishers'],
    'Mit Press': ['MIT PRESS OPEN ACCESS'],
    'MY JOVE CORP': ['My JOVE corporation', 'MYJoVE Corporation'],
    'National Academy of Sciences': 
            ['National Academy of Sciences of the United States of America',
            'National Academy of Sciences USA'],
    'Nature Publishing Group': ['Nature',
                               'Nature PG',
                               'Nature Publishing',
                               'Nature publishing group',
                               'NATURE PUBLISHING GROUP LTD',
                               'NPG'],
'Oxford Journals': ['Oxford Journals (OUP)'],
    'Oxford University Press': ['Oxford Univ Press',
                               'OXFORD UNIVERSITY PRESS',
                               'Oxford University Press (OUP)',
                               'Oxford Univesity Press'],
    'PLoS (Public Library of Science)': ['PLoS', 'Plos', 'PLOS', 'Public Library of Science'],
    'PNAS': ['PNAS Author Publication'],
    'Portland Press': ['Portland press', 
                       'PORTLAND PRESS LTD', 
                       'Portland Press Ltd'],
    'PubMed': ['PubMed Central'],
    'Royal Society': ['Royal Society for Chemistry', 
                      'Royal Society of Chemistry',
                      'RSC', 
                      'RSC Publishing', 
                      'The Royal Society', 
                      'The royal Society'],
    'Sage Publications': ['Sage', 
                          'SAGE', 
                          'SAGE Publications', 
                          'Sage Publications Inc',
                          'Sage Publications Ltd', 
                          'Sage Publishers',
                          'Sage Publishing'],
    'Society for General Microbiology': ['Society for Genermal Microbiology'],
    'Society for Neuroscience': ['Society for Neurosciences',
                                'SOCIETY OF NEURO SCIENCES',
                                'Society of Neuroscience'],
    'Springer': ['Springer - Verlag GMBH', 
                 'Springer Science + Business Media',
                 'Springer-Veriag GmbH', 
                 'SPRINGER-VERLAG GMBH',
                 'Springer-Verlag GmbH', 
                 'Springer-Verlag GmbH',
                 'Springer-Verlag GMBH & Ci',
                 'Springer-Verlag GmbH', 
                 'Heidelberger Platz 3, D-14197 Berlin'],
    'Taylor & Francis': ['T&F',
                        'Taylor & Francis Journals '],
    'The Endocrine Society': ['The Endocrine Socety',
                             'THE ENDOCRINE SOCIETY',
                             'The Endrocrine Society'],
    'Wiley': ['Wiley & Son',
             'Wiley Blackwell',
             'Wiley Online Library',
             'Wiley Subscription Services',
             'Wiley Subscription Services  Inc.',
             'Wiley Subscription Services  Inc'
             'Wiley Subscription Serviices Inc',
             'Wiley VCH',
             'Wiley-Blackwell',
             'Wiley-Blackwell, John Wiley & Sons',
             'Wiley-VCH',
             'Wliey-Blackwell'],
    'Wolters Kluwer': ['Wolters Kluwer Health',
                      'Wolters Kluwer N.V./Lippinott',
                      'Wolters Kluwers'] 
}

for publisher in df.Publisher:
    for k, v in publisher_changes.items():
        if publisher in v:
            df.Publisher[df.Publisher == publisher] = k

In [19]:
# Data cleaning for Journal column

journal_changes = {
    'Journal of Medicinal Chemistry': ['J Med Chem',
                                      'J Med Chem.'],
    'ACS Nano': ['ACS NANO'],
    'Journal of Neurophysiology': ['The Journal of Neurophysiology'],
    'American Journal of Psychiatry': ['Americal Journal of Psychiatry'],
    'Journal of Biological Chemistry': ['J Biol Chem.',
                                       'JOURNAL OF BIOLOGICAL CHEMISTRY',
                                       'The Journal of Biological Chemistry',
                                       'J Biol Chemistry',
                                       'Journal of biological chemistry',
                                       'The journal of Biological Chemistry',
                                       'Jnl Biological Chemistry',
                                       'Journal Biological Chemistry',
                                       'Journal of Biol Chem',
                                       'Biochem Journal',
                                       'Jnl Biological Chemistry'],
    'Antimicrobial Agents and Chemotherapy': ['Antimicrobial Agfents and Chemotherapy',
                                             'Antimicobial Agents and Chemotherapy'], # typo
    'Blood': ['Blood Journal 2012'],
    'Journal of Clinical Microbiology': ['Jounral of Clinical Microbiology',
                                        'J Clin Microbiol'], # typo
    'Molecular and Cellular Proteomics': ['MCP (Molecular & Cellular Proteomics)'],
    'BMC Genomics': ['BMC Genome Biology'],
    'International Journal of Behavioral Nutrition and Physical Activity':
            ['International Journal of Behavioural Nutrition and Physical Activity'],
    'European Journal of Endocrinology': ['Journal of Endocrinology'],
    'BMJ': ['BMJ Open',
           'BMJ OPEN',
           'BMJ open'],
    'British Journal of Ophthalmology': ['British Journal of Opthalmology'],
    'Journal of Neurology, Neurosurgery & Psychiatry ': 
            ['Journal of Neurology, Neurosurgery and Psychiatry'],
    'Sexually Transmitted Infections': ['Sex Transm Infect.'],
    'British Journal for the History of Science':
        ['British Journal of the History of Science'],
    'Psychological Medicine': ['Pyschological Medicine'],
    'Journal of Cell Science': ['J Cell Sci.'],
    'American Journal for Clinical Nutrition': 
            ['American Journal of Clinical Nutrition'],
'Cell Reports': ['Cell Press - Cell Reports'],
    'Genetics': ['GENETICS'],
    'Journal of Immunology': ['J Immunol'
                             'The Journal of Immunology'],
    'Behavior Research and Therapy': ['Behaviour Research and Therapy'],
    'Chemistry & Biology': ['Chemsitry & Biology'],
    'Consciousness and Cognition': ['Consciousness & Cognition'],
    'Current Biology': ['CURRENT BIOLOGY'],
    'Current Opinions in Neurobiology': ['Current Opinions in Neurobiology'],
    'Developmental Cell': ['DEVELOPMENT CELL'],
    'FEBS Letters': ['Febs Letters'],
    'Gene Therapy': ['Gene',
                    'GENE'],
    'Health & Place': ['Health and Place'],
    'Immunobiology': ['Immnunobiology'],
    'Journal of Allergy and Clinical Immunology': 
            ['JOURNAL OF ALLERGY & CLINICAL IMMUNOLOGY',
            'Journal of allergy and clinical immunology'],
    'Journal of Behavior Therapy and Experimental Psychiatry':
            ['Journal of Behaviour therapy and Experimental Psychiatry'],
    'Lancet Global Health': ['Lancet'],
    'Molecular and Cellular Endocrinology': ['Molecluar & Cellular Endocrinology'],
    'NeuroImage': ['Neuroimage: Clinical'],
    'Social Science and Medicine': ['Social Science & Medicine'],
    'Studies in History and Philosophy of Science Part C':
            ['Studies in History and Philosophy of Science Part C: Studies in History and Philosophy of Biological and Biomedical Sciences'],
    'The American Journal of Human Genetics': ['The Americal Journal of Human Genetics'],
    'The Lancet Neurology': ['The Lancet'],
    'Trends in Neuroscience': ['Trends in Neurosciences'],
    'FASEB Journal': ['The FASEB Journal'],
    'International Journal of Tuberculosis  and Lung Disease': ['IJTLD'],
    'Acta Crystallographica, Section D': 
            ['Acta Crystallographica Section D: Biological Crystallography',
            'Acta Crystallography D',
            'Acta D'],
    'Acta Crystallographica Section F: Structural Biology and Crystallization Communications':
            ['ACTA F'],
    'Epigenetics': ['Epigentics'],
    'Antioxidants & Redox Signaling': ['Antioxidants and Redox Signaling'],
    'The Journal of Cognitive Neuroscience': ['Journal of Cognitive Neuroscience'],
    'Journal of Visualized Experiments': ['Journal of Visulaized expermiments'],
    'Proceedings of the National Academy of Sciences':
            ['Proceddings of the National Academy of Sciences of USA',
            'Proceedings of the National Academy of Sciences of the United States of America'],
    'EMBO Journal': ['EMBO',
                    'The EMBO journal',
                    'The EMBO Journal'],
    'Nature Communications': ['Nature communications',
                             'NATURE COMMUNICATIONS'],
    'Scientific Reports': ['Sci Rep',
                          'Scientific Reports-11-00861B'],
    'American Journal of Epidemiology': ['American Jnl Epidemiology'],
    'International Journal of Epidemiology': ['Int J Epidemiol',
                                             'International Joural of Epidemiology'],
    'Clinical Infectious Diseases': ['Clinicla Infectious Disease'],
    'International Journal of Epidemiology': ['Inyernational Journal of Epidemiology'],
    'Bioinformatics': ['Biinformatics',
                      'Bioinformatics Online'],
    'Nucleic Acids Research': ['NUCLEIC ACIDS RESEARCH'],
    'The Journal of Infectious Diseases': ['J Infect Dis',
                                          'Journal of Infectious Diseases'],
    'PloS One': ['PLoS ONE',
                'PLOS ONE',
                 'PLoS One',
                 'PLoS 1',
                'Plos ONE',
                'PLOS One',
                'PLoSONE',
                'PLosONE'],
    'Proceedings of the National Academy of Sciences': 
                            ['PNAS',
                            'Proceedings of the National Academy of Sciences (PNAS)',
                            'PNAS USA',
                            'Proceedings of the National Academy of Sciences of the USA',
                            'Proc Natl Acad Sci U S A.',
                            'Proceedings of the national Academy of Sciences'],
    'Biochemical Journal': ['Biochemical Journals',
                           'The Biochemical Journal'],
    'PLoS Computational Biology': ['PLOS COMPUTATIONAL BIOLOGY'],
    'PLoS Genetics': ['PLOS Genetics',
                     'PLoS GENETICS'],
    'PLoS Neglected Tropical Diseases': ['PLoS Negected Tropical Diseases'],
    'PLoS Pathogens': ['PLOS Pathogens',
                      'Plos Pathogens'],
    'Public Library of Science': ['Public Library of Science ONE'],
    'British Journal of Psychiatry': ['Britsh Journal of Psychiatry',
                                     'The British Journal of Psychiatry'],
    'Journal of the Royal Society Interface': ['Journal of the Royal Society, Interface'],
    'Proceedings of the Royal Society': ['Proceedings of Royal Society',
                                        'Proceedings of the Royal Society B: Biological Sciences'],
    'Journal of Neuroscience': ['The Journal of Neuroscience'],
    'European Child and Adolescent Psychiatry': 
            ['European Child and Adolescent Psychiatty'],
    'Social Psychiatry and Psychiatric Epidemiology': 
            ['Social Psychiatry and Psychiatric Epidemiol'],
    'Aids Care': ['AIDS Care'],
    'Journal of Clinical Endocrinology and Metabolism':
            ['Endocrinology',
            'Journal of Clinical Endocrinology'],
    'British Journal of Psychiatry': ['BJP'],
    'FEBS Journal': ['FEBS J.',
                    'The FEBS Journal'],
    'British Journal of Pharmacology': ['British Jounal of Pharmacology'],
    'Genetic Epidemiology': ['Genetic Epidemology'],
    'AIDS Journal': ['AIDS'],
    'Journal of Acquired Immune Deficiency Syndromes':
            ['Journal of Acquired Immune Deficiency Syndroms (JAIDS)',
            'JAIDS Journal of Acquired Immune Deficiency Syndromes'],
    'NeuroImage': ['Neuroimage']

}

for journal in df.Journal_Title:
    for k, v in journal_changes.items():
        if journal in v:
            df.Journal_Title[df.Journal_Title == journal] = k

In [21]:
# set row for each iteration in the data frame
row = 0

# cycle through rows in Unnamed columns
# this problem arose because some data was shifted rightward out of the article title
#   and into the price section
for item in df.Unnamed:5,
    # split each item on the currency symbol
    item_wo_currency = str(item).split('£')
    
if item_wo_currency[0] != 'nan':
        # values effected
        print(item_wo_currency)
        print(row)
        
        # use the row iterator to set the column index
        row_to_edit = df.ix[row]
        
        # correctly add the misplaced cost data back to the article title
        row_to_edit.Article_Title = row_to_edit.Article_Title + row_to_edit.Cost
        
        # correctly move the cost data back into the correct row
        row_to_edit.Cost = row_to_edit.Unnamed
        
        # set the needless column to null
        row_to_edit.Unnamed = np.nan
        
        row = row + 1

IndentationError: unexpected indent (<ipython-input-21-553863397e06>, line 9)